In [ ]:
# -- select * from  dsc_dwd.dwd_wh_dsc_inventory_dtl_di 
# -- where src = 'scale'
# -- and data_source in ('scale_hpi', 'scale_michelin', 'scale_fas', 'scale_bose')
# -- and inc_day in ('20211124', '20211117', '20211110', '20211103', 
# -- '20211027', '20211020', '20211013','20211006')
# -- and usage_flag = '1'

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import re
import os
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

print(os.listdir('data_down'))

['inv_1220.csv', 'inv_7.csv', 'inv_7_2.csv']


In [50]:

def allsundays(year):
    return pd.Series(pd.date_range(start=str(year), end=str(year+12), 
                        freq='2W-TUE').strftime('%Y%m%d'))
fridays = tuple([i for i in list(allsundays(2021)[allsundays(2021) < date.today().strftime('%Y%m%d')][-8:])])



In [51]:
fridays

('20210914',
 '20210928',
 '20211012',
 '20211026',
 '20211109',
 '20211123',
 '20211207',
 '20211221')

In [17]:
%%time
df = pd.read_csv('./data_down/inv_1220.csv', sep='\001')
df.columns = [re.sub('\w+\.', '', i) for i in list(df.columns)]
df = df.dropna(how = 'all', axis =1)
time_cols = pd.Series(df.columns)[pd.Series(df.columns).str.lower().str.findall('date|time').apply(len)>0]
df[time_cols] = df[time_cols].apply(lambda x: x.str.slice(0,10))
df9 = df

from datetime import date, timedelta

# sql = """
#     select * from  dsc_dwd.dwd_wh_dsc_inventory_dtl_di 
#     where src = 'scale'
#     and ou_code in (
#     'HPPXXWHWDS', 
#     'MICHETCTGS'
#     'COSTASHHTS',
#     'ZEBRASHALS'
#     )
#     and inc_day in """+ str(fridays) + """
#     and usage_flag = '1'
#     """
# print(sql)

Wall time: 1.72 s


In [28]:
# df['ou_code'] in {'MICHETCTGS', 'HPPXXWHWDS'}
'HPPXXWHWDS' in {'MICHETCTGS', 'HPPXXWHWDS'}

True

In [45]:
code = ''
df0 = pd.DataFrame()
scan_len = len(df['inc_day'].unique())  

def load_data(ou_code):
    """
    load bose data;
    所有类型的qty都要加起来哦
    只选择有多个收货日期的货物
    """
    global code, df, bose_inv, df0
    df0 = pd.DataFrame()
    df = df9
    df = df[df['ou_code'].astype(str) == ou_code]

    def fifo_fefo(df, type):
        if type == 'fifo':
            df['recived_date'] = pd.to_datetime(df['recived_date'])
            df['fifo_fefo'] = 'fifo'
        elif type == 'fefo':
            df['recived_date'] = pd.to_datetime(df['expiration_date'].str.slice(0,10))
            df['fifo_fefo'] = 'fefo'
        else: 
            pass
        return df 

    if ou_code in {'HPPXXWHWDS', 'HPPXXSHMGS'}:
        # hp wh
        code = '(QH|27|QI)'
        df = fifo_fefo(df, 'fifo')

    elif ou_code in {'MICHETCTGS', 'MICHESHXCS'}:
        df = df[df['expiration_date'] != '4712-12-31']
        # mich tc, rt,m  FEFO
        code = '(BLOCKED_TH|RETURN)'
        df = fifo_fefo(df, 'fefo')

    elif ou_code == 'COSTASHHTS':
        # COSTASHHTS expiration_date 没有空值.
        df1 = df[df['expiration_date'] == '4712-12-31'] # fifo @# scale datetime. need redefine when other wms sys/..
        df2 = df[df['expiration_date'] != '4712-12-31'] # fefo
        df1 = fifo_fefo(df1, 'fifo')
        df2 = fifo_fefo(df2, 'fefo')
        df = pd.concat([df1, df2], axis = 0)
        code = '(blocked)'

    elif ou_code == 'SIEMESUEPS':
        pass

    # print(code)
    
    
    df = df[['wms_company_name', 'wms_warehouse_id','sku_code', 'sku_name', 'sku_desc', 'location',\
        'lock_codes', 'on_hand_qty', 'in_transit_qty','allocated_qty', 'shelf_days', 
        'recived_date','usage_flag', 'fifo_fefo','inc_day', 'ou_code']]
    df['qty'] = df['on_hand_qty']
    
    
    # 没有重复的 目前看....aaa
    print("===============================oucode :: %s================================="%df['ou_code'].unique())
    print(df['on_hand_qty'].describe())

    df = df.groupby(
        ['recived_date', 'sku_code', 'lock_codes','inc_day', 'wms_warehouse_id', 'fifo_fefo'],
        # dropna = False
        ).agg(
    {
        'qty':'sum',
        'location': set
    }
    ).sort_values(['sku_code', 'recived_date']).reset_index()
    # 只选择有多个收货日期的货物
    filter0 = df.groupby(['sku_code'])['recived_date'].agg(
    {
        set
    }
        ).reset_index()

    filter0 = pd.DataFrame(filter0[filter0['set'].apply(len)> 1]['sku_code'].drop_duplicates())
    bose_inv = filter0.merge(df, on = ['sku_code'], how = 'inner')\
        .sort_values(['recived_date','sku_code', 'inc_day'])
    bose_inv['ou_code'] = ou_code
    print(bose_inv.head())
    return bose_inv

def snapshot():
    """
    pivot table. inc_day 快照 作为 cols
    添加标记.
    """
    global df0, bose_inv
    df0 = pd.DataFrame()
    for i in bose_inv['sku_code'].unique():
        df_out = bose_inv[bose_inv['sku_code'] == i]\
            .pivot_table(columns=['inc_day'], index = 'recived_date', values=['qty']).reset_index()
        df_out['sku_code'] = i
        df0 = pd.concat([df0, df_out], axis = 0)
    try:
        df0.columns = df0.columns.get_level_values(level=1)
    except:
        pass
    
    df0.columns = list(df0.columns[0:scan_len]) + ['received_date','sku']
    df0 = df0.sort_values(['sku', 'received_date'])
    df0['mark'] = 0
    
    df0['mark'] = df0['mark'].where(
        df0.iloc[:, 0: (scan_len - 1)].isna().all(axis = 1) == False, 'new')
    df0['mark'] = df0['mark'].where(
        ~df0.iloc[:,(scan_len - 1)].isna() , 'clear')
    # fill na~
    df0 = df0.fillna(0)
    bose_inv = bose_inv.rename({'sku_code':'sku', 'recived_date':'received_date'}, axis = 1)\
        .reset_index(drop = True).drop(['inc_day', 'qty'], axis = 1)
    bose_inv = bose_inv.drop_duplicates(subset = ['sku', 'received_date', 'lock_codes'])
    df0 = df0.merge(bose_inv, on = ['sku','received_date'], how = 'left')

    # may lock
    df0['mark'] = df0['mark'].where(df0.iloc[:, 0:scan_len].fillna(0).nunique(axis = 1) > 1, 'may_lock')
    print("===============================snap!done for : %s=================================" %df0['ou_code'].unique())

# print("===============================mid_function_check=================================")
def err_part():
    """
    findout who are the naught peach.
    err 中干掉了 new 干掉了maylock 
    """
    global df0
    print("===============================err_part!start: %s================================="%df0.info())
    
    df_err = df0[df0['mark'] != 'new']
    # 补充可能被锁的标记
    # df_err['mark'] = df_err['mark'].where(df_err.iloc[:, 0:8].fillna(0).nunique(axis = 1) > 1, 'may_lock')
    # 干掉了maylock
    df_err = df_err[df_err['mark'] != 'may_lock'].sort_values(['sku', 'received_date'])
    # print("===============================scan_len_err_function--%s================================="%scan_len)
    # print(df_err.iloc[:,0:scan_len])
    df_err['change'] = df_err.iloc[:,0:scan_len].diff(axis = 1).sum(axis = 1)
    shift = df_err.groupby(['sku', 'wms_warehouse_id']).shift(1) 
    shift = shift[['mark','change']]
    shift.columns = ['lag_mark', 'lag_change']
    shift['lag_mark'] = shift['lag_mark'].where(~shift['lag_mark'].isna(), 'clear')
    df_err = pd.concat([df_err, shift], axis = 1)
    print("===============================err_part!done=================================")

    return df_err

def output(df_err):
    global df0
    print("===============================output!start=================================")

    dishes = list(df_err[(df_err['lag_mark'] != 'clear') \
        & (df_err['change'] < 0)
        & (df_err['change'] != df_err['lag_change'])]['sku'].unique())
    print("===============================output!done=================================")

    return df0[df0['sku'].isin(dishes)]

def check(sku ):
    a = df0[df0['sku'].isin(sku)].sort_values(['sku','received_date'])
    print("===============================check!done=================================")

    return a 

def ou_level_lock_codes(lock_code_to_eliminate):
    """
    正则. lock_code 需要被排除的, 依赖view表格. 
    """ 
    
    select_none_lock  = pd.DataFrame(
        view.groupby('sku')[
            'mark'
            ].apply(list).astype(str).str.match('.+may')
        ).reset_index()

    select_none_lock2 = pd.DataFrame(
        view.groupby('sku')[
            'lock_codes'
            ].apply(list).astype(str).str.match('.+'+lock_code_to_eliminate)
        ).reset_index()
    # 去重    
    bose_err_list = set(select_none_lock[~select_none_lock['mark']]['sku'].unique())
    bose_err_list2 = list(select_none_lock2[~select_none_lock2['lock_codes']]['sku'].unique())
    bose_err_list = list(bose_err_list.intersection(bose_err_list2))
    bose_err_list = list(set(bose_err_list))
    print("===============================ou_level_lock_codes!done=================================")

    return bose_err_list


In [36]:
# # bose_inv = load_data('COSTASHHTS');bose_inv
# # snapshot()
# for i in df9['ou_code'].unique():
#     if i in {'s', 'HPPssXXWHWDS', 'MICHETCTGS'}:
#         print(0)
#     else :
#         pass

In [47]:

out_df = pd.DataFrame()
for ou_code0 in df9['ou_code'].unique():

    bose_inv = load_data(ou_code0)
    
    print('{note:=>50}'.format(note=ou_code0) + '{note:=>50}'.format(note=''))
    print("===============================this_code: %s================================="%code)

    df0 = pd.DataFrame()
    print("===============================boseInv before snap=================================")
    print(bose_inv.info())
    snapshot()

    df0.head()
    print('{note:=>50}'.format(note='~') )
    
    df_err = err_part()
    print('{note:=>50}'.format(note='~')  )

    view = output(df_err)
    bose_err_list = ou_level_lock_codes(code)
    bose_definite_wrong = check(bose_err_list)
    out_df = pd.concat([out_df, bose_definite_wrong], axis = 0)

===============================oucode :: ['COSTASHHTS']=================================
count     8921.000000
mean       268.981372
std       1284.248128
min          0.000000
25%          4.000000
50%         22.000000
75%        141.000000
max      44235.000000
Name: on_hand_qty, dtype: float64
         sku_code recived_date lock_codes   inc_day wms_warehouse_id  \
1688  COSTA050174   2015-08-10  Available  20211203         COSTA_SH   
1689  COSTA050174   2015-08-10  Available  20211210         COSTA_SH   
1690  COSTA050174   2015-08-10  Available  20211217         COSTA_SH   
1694  COSTA050175   2015-08-10  Available  20211203         COSTA_SH   
1695  COSTA050175   2015-08-10  Available  20211210         COSTA_SH   

     fifo_fefo    qty      location     ou_code  
1688      fifo  137.0  {D-06-36-03}  COSTASHHTS  
1689      fifo  137.0  {D-06-36-03}  COSTASHHTS  
1690      fifo  137.0  {D-06-36-03}  COSTASHHTS  
1694      fifo   98.0  {D-06-36-03}  COSTASHHTS  
1695      fifo   9

In [48]:
out_df['ou_code'].unique()

array(['COSTASHHTS', 'HPPXXWHWDS', 'MICHETCTGS'], dtype=object)

In [49]:
out_df.head(2)

,20211203,20211210,20211217,received_date,sku,mark,lock_codes,wms_warehouse_id,fifo_fefo,location,ou_code
111,50.0,10.0,0.0,2022-01-06,COSTA020760,clear,Available,COSTA_SH,fefo,"{D-115-05-03-A, SHIP-01}",COSTASHHTS
112,1003.0,0.0,0.0,2022-01-13,COSTA020760,clear,Available,COSTA_SH,fefo,"{D-115-05-03-A, SHIP-01}",COSTASHHTS


In [13]:
# out_df[out_df['ou_code'] == 'COSTASHHTS']

In [ ]:
out_df['lock_codes'].unique()

In [ ]:
def load_data(ou_code):
    """
    load bose data;
    所有类型的qty都要加起来哦
    只选择有多个收货日期的货物
    """
    global code
    df = df9
    df = df[df['ou_code'].astype(str) == ou_code]

    def fifo_fefo(df, type):
        if type == 'fifo':
            df['recived_date'] = pd.to_datetime(df['recived_date'])
            df['fifo_fefo'] = 'fifo'
        elif type == 'fefo':
            df['recived_date'] = pd.to_datetime(df['expiration_date'])
            df['fifo_fefo'] = 'fefo'
        else: 
            pass
        return df 

    if ou_code == 'HPPXXWHWDS':
        # hp wh
        code = '(QH|27|QI)'
        df = fifo_fefo(df, 'fifo')

    elif ou_code == 'MICHETCTGS':
        # mich tc, rt,m  FEFO
        df = fifo_fefo(df, 'fefo')

    elif ou_code == 'COSTASHHTS':
        # COSTASHHTS expiration_date 没有空值.
        df1 = df[df['expiration_date'] == '4712-12-31 00:00:00'] # fifo
        df2 = df[df['expiration_date'] != '4712-12-31 00:00:00'] # fefo
        df1 = fifo_fefo(df1, 'fifo')
        df2 = fifo_fefo(df2, 'fefo')
        df = pd.concat([df1, df2], axis = 0)

    elif ou_code == 'SIEMESUEPS':
        pass
    
    
    df = df[['wms_company_name', 'wms_warehouse_id','sku_code', 'sku_name', 'sku_desc', 'location',\
        'lock_codes', 'on_hand_qty', 'in_transit_qty','allocated_qty', 'shelf_days', 
        'recived_date','usage_flag', 'fifo_fefo','inc_day']]
    # qty = pd.Series(df.columns).str.extract('(.+qty)').dropna(axis = 0)
    # df['qty'] = df[qty[0]].sum(axis = 1).round(2)
    df['qty'] = df['on_hand_qty']
    
    # bose_inv.drop_duplicates(subset = ['sku_code', 'recived_date', 'lock_codes', 'inc_day', 'qty'])
    # 没有重复的 目前看....aaa
    df = df.groupby(
        ['recived_date', 'sku_code', 'lock_codes','inc_day', 'wms_warehouse_id', 'fifo_fefo'], dropna = False
        ).agg(
    {
        'qty':sum,
        'location': set
    }
    ).sort_values(['sku_code', 'recived_date']).reset_index()
    # 只选择有多个收货日期的货物
    filter0 = df.groupby(['sku_code'])['recived_date'].agg(
    {
        set
    }
        ).reset_index()

    filter0 = pd.DataFrame(filter0[filter0['set'].apply(len)> 1]['sku_code'].drop_duplicates())
    bose_inv = filter0.merge(df, on = ['sku_code'], how = 'inner')\
        .sort_values(['recived_date','sku_code', 'inc_day'])
    
    return bose_inv



In [ ]:
# =======================
# # 只选择多次出库的货物
# filter = bose.groupby(['recived_date', 'sku_code'])['inc_day'].agg({set}).reset_index()
# filter = filter[filter['set'].apply(len)> 1][['recived_date','sku_code']].drop_duplicates();filter
# bose_inv.head(4)
bose_inv = load_data('HPPXXWHWDS')

In [ ]:
%%time
df0 = pd.DataFrame()
def snapshot():
    """
    pivot table. inc_day 快照 作为 cols
    添加标记.
    """
    global df0, bose_inv
    for i in tqdm(bose_inv['sku_code'].unique(), ascii= False, colour = 'green'):
        df_out = bose_inv[bose_inv['sku_code'] == i]\
            .pivot_table(columns=['inc_day'], index = 'recived_date', values=['qty']).reset_index()
        df_out['sku_code'] = i
        df0 = pd.concat([df0, df_out], axis = 0)
    try:
        df0.columns = df0.columns.get_level_values(level=1)
    except:
        pass
    df0.columns = list(df0.columns[0:8]) + ['received_date','sku']
    df0 = df0.sort_values(['sku', 'received_date'])
    df0['mark'] = 0
    # 这个吊东`西不知道为什么是反选的 
    df0['mark'] = df0['mark'].where(df0.iloc[:, 0:7].isna().all(axis = 1) == False, 'new')
    df0['mark'] = df0['mark'].where(~df0.iloc[:,7].isna() , 'clear')
    # fill na~
    df0 = df0.fillna(0)
    bose_inv = bose_inv.rename({'sku_code':'sku', 'recived_date':'received_date'}, axis = 1)\
        .reset_index(drop = True).drop(['inc_day', 'qty'], axis = 1)
    bose_inv = bose_inv.drop_duplicates(subset = ['sku', 'received_date', 'lock_codes'])
    df0 = df0.merge(bose_inv, on = ['sku','received_date'], how = 'left')

    # may lock
    df0['mark'] = df0['mark'].where(df0.iloc[:, 0:8].fillna(0).nunique(axis = 1) > 1, 'may_lock')
snapshot()

In [ ]:
def err_part():
    """
    findout who are the naught peach.
    err 中干掉了 new 干掉了maylock 
    """
    df_err = df0[df0['mark'] != 'new']
    # 补充可能被锁的标记
    # df_err['mark'] = df_err['mark'].where(df_err.iloc[:, 0:8].fillna(0).nunique(axis = 1) > 1, 'may_lock')
    # 干掉了maylock
    df_err = df_err[df_err['mark'] != 'may_lock'].sort_values(['sku', 'received_date'])
    df_err['change'] = df_err.iloc[:,0:8].diff(axis = 1).sum(axis = 1)
    shift = df_err.groupby(['sku', 'wms_warehouse_id']).shift(1) 
    # q = []
    # for i in shift['mark']:
    #     if pd.isna(i):
    #         q.append('clear')
    #     elif i =='clear':
    #         q.append('clear')
    #     else:
    #         q.append('check')
    # shift['mark'] = q
    # shift['mark2'] = shift.iloc[:,0:8].diff(axis = 1).sum(axis = 1)
    shift = shift[['mark','change']]
    shift.columns = ['lag_mark', 'lag_change']
    shift['lag_mark'] = shift['lag_mark'].where(~shift['lag_mark'].isna(), 'clear')
    df_err = pd.concat([df_err, shift], axis = 1)
    # df_wrong = df_err[(df_err['lag_mark'] != 'clear') & (df_err['change'] != 0)]
    # df_good  = df_err[(df_err['lag_mark'] != 'clear') | (df_err['change'] != 0)]
    return df_err
df_err = err_part()

In [ ]:
# df_err.to_pickle('./data_up/df_err_bose.p')
# df0.to_pickle('./data_up/df0_bose.p')
# df_err_bose = pd.read_pickle('./data_up/df_err_bose.p')
# df0_bose = pd.read_pickle('./data_up/df0_bose.p')
# # df0_bose = df0_bose[df0_bose['wms_warehouse_id'] == 'BOSE_SH']
# # df_err_bose = df_err_bose[df_err_bose['wms_warehouse_id'] == 'BOSE_SH']

In [ ]:

def output(df_err, df0):
    """
    筛选lag_mark, 
    """
    dishes = list(df_err[(df_err['lag_mark'] != 'clear') \
        & (df_err['change'] < 0)
        & (df_err['change'] != df_err['lag_change'])]['sku'].unique())
    print(df0['lock_codes'].unique())
    return df0[df0['sku'].isin(dishes)]
def check(sku, df0):
    return df0[df0['sku'].isin(sku)].sort_values(['sku','received_date'])


view = output(df_err,df0)

def ou_level_lock_codes(lock_code_to_eliminate):
    """
    正则. lock_code "需要"被排除的, 依赖view表格. 
    """
    # output(df_err_bose,df0_bose)['sku'].unique()
    select_none_lock  = pd.DataFrame(
        view.groupby('sku')['mark'].apply(list).astype(str).str.contains('may') # may lock?
        ).reset_index()
    select_none_lock2 = pd.DataFrame(
        view.groupby('sku')['lock_codes'].apply(list).astype(str).str.contains(lock_code_to_eliminate)
        ).reset_index()
    # 去重    
    bose_err_list = set(select_none_lock[~select_none_lock['mark']]['sku'].unique())
    bose_err_list2 = list(select_none_lock2[~select_none_lock2['lock_codes']]['sku'].unique())
    bose_err_list = list(bose_err_list.intersection(bose_err_list2))
    bose_err_list = list(set(bose_err_list))
    
    return bose_err_list



In [ ]:
# pd.DataFrame(
#         view.groupby('sku')['mark'].apply(list).astype(str).str.contains('may') # may lock?
#         ).reset_index()

bose_inv['lock_codes'].unique()

In [ ]:

# 1000全新 3000退货 4000破损产品-

bose_err_list = ou_level_lock_codes('(QH|QI|27)')
bose_definite_wrong = check(bose_err_list, df0)
bose_definite_wrong['lock_codes'].unique()
    # .to_csv('./data_up/bose_fifo.csv', index = None, encoding = 'utf_8_sig')

=================================
# HP
=================================


In [ ]:
%%time
# del df0 ,bose_inv
bose_inv = load_data('HPI_WH');bose_inv
df0 = pd.DataFrame()
snapshot()

# snapshot()
df_err = err_part()

In [ ]:
# df0_hp['lock_codes'].unique()

In [ ]:
df_err.to_pickle('./data_up/df_err_hpwh.p')
df0.to_pickle('./data_up/df0_hpwh.p')
df_err_hp = pd.read_pickle('./data_up/df_err_hpwh.p')
df0_hp = pd.read_pickle('./data_up/df0_hpwh.p')


In [ ]:
view = output(df_err_hp, df0_hp)
err_list_hp = ou_level_lock_codes('(QH|27)')

In [ ]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0_hp)

# Z7Y79A 问一下
# 
hp_definite_wrong.head(6)

In [ ]:
# hp_definite_wrong[hp_definite_wrong['wms_warehouse_id']=='HPI_WH']

In [ ]:
hp_definite_wrong.to_csv('./data_up/hpwh_fifo.csv', index = None, encoding = 'utf_8_sig')

In [ ]:
# hp_definite_wrong[hp_definite_wrong['sku'] == '1VV21AA']

In [ ]:
hp_definite_wrong.to_csv('./data_up/hpsh_fifo.csv', index = None, encoding = 'utf_8_sig')

=================================
# michelin
=================================


In [ ]:
df0 = []
bose_inv = []

In [ ]:
df9[df9['wms_warehouse_name'] == 'OE']

In [ ]:
%%time
# del df0 ,bose_inv
bose_inv = load_data('RT');bose_inv
df0 = pd.DataFrame()
snapshot()

# snapshot()
df_err = err_part()

In [ ]:
df_err.to_pickle('./data_up/df_err_michrt.p')
df0.to_pickle('./data_up/df0_michrt.p')
df_err_mich = pd.read_pickle('./data_up/df_err_michrt.p')
df0_mich = pd.read_pickle('./data_up/df0_michrt.p')


In [ ]:
output(df0=df0_mich, df_err=df_err_mich).head(3)

In [ ]:
# df0_mich['lock_codes'].str.slice(0,7).unique()

In [ ]:
view = output(df0=df0_mich, df_err=df_err_mich)
err_list_mich = ou_level_lock_codes('(BLOCKED_TH|RETURN)')

In [ ]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]

mich_definite_wrong = check(err_list_mich, df0 = df0_mich)

# MATMICHELIN00591

# 同一时间 received 货物为什么会增加
# mich dot date?

In [ ]:
df0_mich[df0_mich['sku'] == '085753_111']

In [ ]:
mich_definite_wrong[mich_definite_wrong['sku'] == '085753_111']

In [ ]:
mich_definite_wrong.to_csv('./data_up/michrt_fifo.csv', index = None, encoding = 'utf_8_sig')

In [ ]:
%%time
# del df0 ,bose_inv
bose_inv = load_data('OE');bose_inv
df0 = pd.DataFrame()
snapshot()
df_err = err_part()
view = output(df0=df0, df_err=df_err)
err_list_mich = ou_level_lock_codes('(BLOCKED_TH|RETURN)')
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]

mich_definite_wrong = check(err_list_mich, df0 = df0)

mich_definite_wrong.head()

In [ ]:
mich_definite_wrong.to_csv('./data_up/michoe_fifo.csv', index = None, encoding = 'utf_8_sig')

==========================
# fas <br>
==========================

In [ ]:
bose_inv = load_data('FAS') 
df0 = pd.DataFrame()
snapshot()

In [ ]:
df_err = err_part()
view = output(df_err, df0)

In [ ]:
bose_inv.head(3)

In [ ]:
err_list_hp = ou_level_lock_codes('(Avi)') 
bose_inv[bose_inv['sku'] == err_list_hp[0]]

In [ ]:

def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0)

# hp_definite_wrong.head(6)

In [ ]:
hp_definite_wrong.to_csv('./data_up/fas.csv', index = None, encoding = 'utf_8_sig')

==========================
# part2 <br>
==========================

**select * from  dsc_dwd.dwd_wh_dsc_inventory_dtl_di 
where src = 'scale'
and  ou_code in(
'FERRECDXXS',
'APPLESHWHW',
'HUSQVSHMFS',
'COSTASHHTS',
'FUJIXSYXXS',
'ZEBRASHALS'
)
and inc_day in ('20211124', '20211117', '20211110', '20211103', 
'20211027', '20211020', '20211013','20211006')
and usage_flag = '1'**

In [ ]:
%%time
df = pd.read_csv('./data_down/inv_7_2.csv', sep='\001')
df.columns = [re.sub('\w+\.', '', i) for i in list(df.columns)]
df = df.dropna(how = 'all', axis =1)
time_cols = pd.Series(df.columns)[pd.Series(df.columns).str.lower().str.findall('date|time').apply(len)>0]
df[time_cols] = df[time_cols].apply(lambda x: x.str.slice(0,10))
df9 = df

## Ferrero

In [ ]:
bose_inv = load_data('FERRECDXXS') 
df0 = pd.DataFrame()
snapshot()
df_err = err_part()
view = output(df_err, df0)

In [ ]:
err_list_hp = ou_level_lock_codes('Blocked') 
bose_inv[bose_inv['sku'] == err_list_hp[0]]

In [ ]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0)

# hp_definite_wrong.head(6)

In [ ]:
hp_definite_wrong.to_csv('./data_up/ferrero_FERRECDXXS.csv', index = None, encoding = 'utf_8_sig')

## Apple sh

In [ ]:
df.ou_code.unique()

In [ ]:
bose_inv = load_data('APPLESHWHW') 
df0 = pd.DataFrame()
snapshot()
df_err = err_part()
view = output(df_err, df0)

In [ ]:
err_list_hp

In [ ]:
err_list_hp = ou_level_lock_codes('ttt') 
# bose_inv[bose_inv['sku'] == err_list_hp[0]]

In [ ]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0)

# hp_definite_wrong.head(6)

In [ ]:
# hp_definite_wrong[hp_definite_wrong['sku'] == '339S00668']

In [ ]:
hp_definite_wrong.to_csv('./data_up/apple_sh_APPLESHWHW.csv', index = None, encoding = 'utf_8_sig')

## costa 

In [ ]:
bose_inv = load_data('COSTASHHTS') 
df0 = pd.DataFrame()
snapshot()
df_err = err_part()
view = output(df_err, df0)

In [ ]:
err_list_hp = ou_level_lock_codes('Held') 
df0[df0['sku'] == err_list_hp[0]]

In [ ]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0)

# hp_definite_wrong.head(6)

In [ ]:
hp_definite_wrong.to_csv('./data_up/costa_COSTASHHTS.csv', index = None, encoding = 'utf_8_sig')

## ZEBRASHALS

In [ ]:
bose_inv = load_data('ZEBRASHALS') 
df0 = pd.DataFrame()
snapshot()
df_err = err_part()
view = output(df_err, df0)

In [ ]:
err_list_hp = ou_level_lock_codes('HOLD1') 
df0[df0['sku'] == err_list_hp[0]]

In [ ]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0)

hp_definite_wrong.head(6)

In [ ]:
hp_definite_wrong.to_csv('./data_up/zebra_ZEBRASHALS.csv', index = None, encoding = 'utf_8_sig')